In [ ]:
pip install pyvi

     |████████████████████████████████| 8.5 MB 3.7 MB/s 
     |████████████████████████████████| 743 kB 64.3 MB/s 


In [ ]:
# Filter label 1 - train

# Dataset analysis
import pandas as pd 

DATA = 'drive/MyDrive/CODE/ViSocial/dataset/ctsd/UIT-ViCTSD_train.csv'
DATA_HATE = 'drive/MyDrive/CODE/ViSocial/dataset/ctsd/aug/data_label_1.csv'

data = pd.read_csv(DATA, index_col=False)

label1 = data.loc[data['Toxicity']==1]

data_new = pd.DataFrame(label1, index=None, columns=['Comment', 'Toxicity'])

print(data_new)
data_new.to_csv(DATA_HATE, header=False, index=False, sep="|")

                                                Comment  Toxicity
2     tôi thấy người lái xe hơi bấm còi mới là người...         1
16                       Ở đâu cũng có những kẻ dở hơi!         1
23    Đọc tin mà thấy căm hờnMặt người dạ thú chẳng ...         1
31    Cái đầm cửa Hà kiều anh trông nhạt nhẽo ...         1
41    Học đại học còn có mấy bạn vô ý thức, quan trọ...         1
...                                                 ...       ...
6966  Có vẻ đẹp nhưng hơi...nhăn nhở .Bình thường có...         1
6986  Đọc mà không kìm nổi sự giận dữ trước hành vi ...         1
6993  Anti toàn lôi cái kq 8-2 ra nói nhưng mà kiểu ...         1
6994               Ác quá ác ! Cho tù khổ sai cho nhớ !         1
6999            2 kẻ này phải xử lý mạnh. Thật căm phẫn         1

[759 rows x 2 columns]


In [ ]:
# TRAIN 
# CODE augmetation: https://github.com/jasonwei20/eda_nlp

import random
from random import shuffle

random.seed(1)
import json


# import nltk

# nltk.download('wordnet')

class Argument:
    input = "drive/MyDrive/CODE/ViSocial/dataset/ctsd/aug/data_label_1.csv"
    output = "drive/MyDrive/CODE/ViSocial/dataset/ctsd/aug/augmented_dataset.txt"
    num_aug = 10
    alpha = 0.15


args = Argument()

# stop words list
stop_words = []
with open("drive/MyDrive/CODE/ViSocial/vietnamese-stopwords.txt", "r") as f:
    stop_words = []
    for line in f:
        dd = line.strip('\n')
        stop_words.append(dd)

# cleaning up text
import re


def get_only_chars(line):
    # clean_line = ""

    # line = line.replace("’", "")
    # line = line.replace("'", "")
    # line = line.replace("-", " ") #replace hyphens with spaces
    # line = line.replace("\t", " ")
    # line = line.replace("\n", " ")
    # line = line.lower()

    # for char in line:
    #     if char in 'qwertyuiopasdfghjklzxcvbnm ':
    #         clean_line += char
    #     else:
    #         clean_line += ' '

    # clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    # if clean_line[0] == ' ':
    #     clean_line = clean_line[1:]
    # return clean_line
    return line


########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################

# for the first time you use wordnet
# import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet


def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            # print("replaced", random_word, "with", synonym)
            num_replaced += 1
        if num_replaced >= n:  # only replace up to n words
            break

    # this is stupid but we need it, trust me
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return new_words


# def get_synonyms(word):
# 	synonyms = set()
# 	for syn in wordnet.synsets(word):
# 		for l in syn.lemmas():
# 			synonym = l.name().replace("_", " ").replace("-", " ").lower()
# 			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
# 			synonyms.add(synonym)
# 	if word in synonyms:
# 		synonyms.remove(word)
# 	return list(synonyms)

def get_synonyms(word):
    synonyms = set()
    with open("drive/My Drive/CODE/HSD/word_net_vi.json", "r") as f:
        wordnet = json.load(f)

    for key, value in wordnet.items():
        if key.strip() == word:
            for v in value:
                synonyms.add(v.strip())

        if word in synonyms:
            synonyms.remove(word)
    return list(synonyms)


########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

def random_deletion(words, p):
    # obviously, if there's only one word, don't delete it
    if len(words) == 1:
        return words

    # randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    # if you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return new_words


########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        if len(new_words) > 0:
            new_words = swap_word(new_words)
    return new_words


def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words


########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words


def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1 and len(new_words) > 0:
    # while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words) - 1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    
    if len(new_words) > 0:
        random_synonym = synonyms[0]
        random_idx = random.randint(0, len(new_words) - 1)
        new_words.insert(random_idx, random_synonym)


########################################################################
# main data augmentation function
########################################################################

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_chars(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word is not '']
    num_words = len(words)

    augmented_sentences = []

    if len(words) <= 0:
        return augmented_sentences
    num_new_per_technique = int(num_aug / 4) + 1
    n_sr = max(1, int(alpha_sr * num_words))
    n_ri = max(1, int(alpha_ri * num_words))
    n_rs = max(1, int(alpha_rs * num_words))

    # sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))

    # ri
    for _ in range(num_new_per_technique):
        a_words = random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))

    # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(' '.join(a_words))

    # rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(' '.join(a_words))

    augmented_sentences = list(set(augmented_sentences))
    augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
    shuffle(augmented_sentences)

    # trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    # append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences


# the output file
output = None
if args.output:
    output = args.output
else:
    from os.path import dirname, basename, join

    output = join(dirname(args.input), 'eda_' + basename(args.input))

# number of augmented sentences to generate per original sentence
num_aug = 9  # default
if args.num_aug:
    num_aug = args.num_aug

# how much to change each sentence
alpha = 0.1  # default
if args.alpha:
    alpha = args.alpha


# generate more data with standard augmentation
def gen_eda(train_orig, output_file, alpha, num_aug=9):
    try:
        writer = open(output_file, 'w')
        lines = open(train_orig, 'r').readlines()

        writer.write("Comment" + "," + "Toxicity" + '\n')
        augm = ""
        for i, line in enumerate(lines):
            try:
                parts = line[:-1].split('|')
                # print(parts)
                # sen_id = parts[0]
                label = parts[1]
                sentence = parts[0]
                aug_sentences = eda(sentence, alpha_sr=alpha, alpha_ri=alpha, alpha_rs=alpha, p_rd=alpha, num_aug=num_aug)
                for aug_sentence in aug_sentences:
                    # writer.write(label + "\t" + aug_sentence + '\n')
                    # writer.write(sen_id + "," +aug_sentence + "," + label + '\n')
                    augm = augm + aug_sentence + "," + label + '\n'
            except Exception as e:
                print(e)
                print(parts)
                pass

        writer.write(augm)
        writer.close()
        print(
            "generated augmented sentences with eda for " + train_orig + " to " + output_file + " with num_aug=" + str(
                num_aug))
    except Exception as e:
        raise e
        pass


# main function
if __name__ == "__main__":
    # generate augmented sentences and output into a new file
    gen_eda(args.input, args.output, alpha=alpha, num_aug=num_aug)


generated augmented sentences with eda for drive/MyDrive/CODE/ViSocial/dataset/ctsd/aug/data_label_1.csv to drive/MyDrive/CODE/ViSocial/dataset/ctsd/aug/augmented_dataset.txt with num_aug=10


In [7]:
# TRAIN
# concat train original with original

import pandas as pd
import csv

DATA = 'drive/MyDrive/CODE/ViSocial/dataset/ctsd/UIT-ViCTSD_train.csv'
# DATA = 'drive/My Drive/CODE/Hate speech detection/data/vlsp_train.csv'
DATA_AUG = 'drive/MyDrive/CODE/ViSocial/dataset/ctsd/aug/augmented_dataset.txt'

# DATA_AUG_FINAL = 'drive/My Drive/CODE/Hate speech detection/data_aug/original_augmented_dataset.csv'
DATA_AUG_FINAL = 'drive/MyDrive/CODE/ViSocial/dataset/ctsd/aug/train_augmented_dataset.csv'

data = pd.read_csv(DATA, index_col=False, usecols=["Comment", "Toxicity"])
data_hate = pd.read_csv(DATA_AUG, index_col=False, error_bad_lines=False)

# data_hate = data_hate.iloc[: , 1:]
data_hate.drop_duplicates(subset ="Comment", keep = False, inplace = True)
data_hate.dropna(subset = ["Toxicity"], inplace=True)

data_aug = pd.concat([data, data_hate])
data_aug.to_csv(DATA_AUG_FINAL, index=False)

b'Skipping line 10: expected 2 fields, saw 3\nSkipping line 46: expected 2 fields, saw 3\nSkipping line 47: expected 2 fields, saw 3\nSkipping line 48: expected 2 fields, saw 3\nSkipping line 49: expected 2 fields, saw 3\nSkipping line 50: expected 2 fields, saw 3\nSkipping line 51: expected 2 fields, saw 3\nSkipping line 52: expected 2 fields, saw 3\nSkipping line 53: expected 2 fields, saw 3\nSkipping line 54: expected 2 fields, saw 3\nSkipping line 55: expected 2 fields, saw 3\nSkipping line 56: expected 2 fields, saw 3\nSkipping line 68: expected 2 fields, saw 3\nSkipping line 70: expected 2 fields, saw 3\nSkipping line 71: expected 2 fields, saw 3\nSkipping line 72: expected 2 fields, saw 3\nSkipping line 73: expected 2 fields, saw 3\nSkipping line 74: expected 2 fields, saw 3\nSkipping line 75: expected 2 fields, saw 3\nSkipping line 76: expected 2 fields, saw 3\nSkipping line 77: expected 2 fields, saw 3\nSkipping line 78: expected 2 fields, saw 3\nSkipping line 90: expected 2 f